In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm

In [2]:
pages_df = pd.read_csv('data/DFKV_pages.csv')

In [3]:
def is_bnf(link):
    return 'gallica.bnf.fr' in link

def is_dub(link):
    return 'digi.ub.uni-heidelberg' in link

In [4]:
PREFIX_URL_GALLICA = 'https://gallica.bnf.fr/iiif/ark:/'
SUFFIX_URL_IMAGE_GALLICA = '/full/pct:50/0/native.jpg' # pct:50 because we download the images with lower quality, to gain storage space

# function that from the canvas link create the image link
def modify_url(link):
    try:
        if is_bnf(link):
            ml = PREFIX_URL_GALLICA + link.split('/')[5] + '/' + link.split('/')[6] + '/' + link.split('/')[8] + SUFFIX_URL_IMAGE_GALLICA
            return ml
        elif is_dub(link):
            response = requests.get(link) # Request the canvas infos.json
            if response.status_code == 200:
                rep = json.loads(response.content)
                im_link = rep['images'][0]['resource']['@id']
                im_link = im_link.replace('/4/', '/1/') # Change to lower image quality
                return im_link
        else:
            return ''
    except:
        return '' # when the url is not conform, just ignore it - it only happens twice

In [5]:
# Function to get the image from the link and save it with the right name at the right place
def download_image(link, doc_id, page):
    response = requests.get(link) # Request the image
    if response.status_code == 200:
        # If request successful, then save the file
        im_path = "./data/DFKV_images/DFKV_" + str(doc_id) + "_" + str(page) + ".jpg"
        file = open(im_path, "wb")
        file.write(response.content) 
        file.close()

In [6]:
pages_df['liens iiif'] = pages_df.apply(lambda row: modify_url(row['liens iiif']), axis=1)

In [14]:
pages_df = pages_df.drop(pages_df[pages_df["liens iiif"] == ''].index)

In [15]:
pages_df.to_csv('data/link_images.csv', index=False)

In [6]:
pages_df = pd.read_csv('data/link_images.csv')

In [ ]:
# Iterate over all the document to download the images
for doc in tqdm(pages_df.iterrows()):
    try:
        page = int(doc[1]['liens iiif'].split('/')[7][1:]) if is_bnf(doc[1]['liens iiif']) else int(doc[1]['liens iiif'].split('/')[7][:-4])
        doc_id = doc[1]['ID']
        # For each document, go through all the desired pages
        for i in range(page, page + int(doc[1]['pages_to_extract'])+1):
            link = doc[1]['liens iiif'].replace(str(page), str(i)) # we change the page number here 
            download_image(link, doc_id, i)
    except:
        continue

25it [01:29,  2.32s/it]